*This notebook gets the raw samknows traceroute measurement data from `curr_traceroute` table; prunes the data to fetch (probeid, timestamp, h1_latencies, h2_latencies) then inserts entire dataframe in `sk_latencies_agg` table.*
- - - 

In [40]:
import sqlite3
import pandas as pd
import requests
import ipaddress
import time

In [41]:
DB_LOCATION = 'lastmile.db'
CURR_TRACEROUTE = 'curr_traceroute'
SK_LATENCIES_AGG = 'sk_latencies_agg'

In [53]:
con = sqlite3.connect(DB_LOCATION)

In [43]:
query = '''SELECT    unit_id as probeid
                   , dtime as timestamp
                   , GROUP_CONCAT(hop, ';') as hop_no
                   , GROUP_CONCAT(rtt_avg, ';') as rtts_h1_h2
                   , '' as h1_avg
                   , '' as h2_avg
           FROM      %s
           WHERE     hop IN (1, 2)
           GROUP BY  probeid, timestamp
        '''%(CURR_TRACEROUTE)

df = pd.read_sql(query, con)

In [44]:
df.head()

,probeid,timestamp,hop_no,rtts_h1_h2,h1_avg,h2_avg
0,14190,2014-07-28 05:42:57,2,1211,,
1,14190,2014-07-28 05:43:01,1;2,377;1280,,
2,14190,2014-07-28 13:41:21,1;2,424;1383,,
3,14190,2014-07-28 13:41:24,1;2,362;1669,,
4,14190,2014-07-28 17:43:23,1;2,395;1297,,


In [45]:
def get_h1_rtt(rtts): 
    try: h1, h2 = rtts.split(';'); return float(h1)/1000
    except Exception as e: print('*', end=''); return None
def get_h2_rtt(rtts): 
    try: h1, h2 = rtts.split(';'); return float(h2)/1000
    except Exception as e: print('*', end=''); return None

df['h1_avg'] = df['rtts_h1_h2'].apply(get_h1_rtt)
df['h2_avg'] = df['rtts_h1_h2'].apply(get_h2_rtt)

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [46]:
del df['hop_no']
del df['rtts_h1_h2']

In [47]:
df.head()

,probeid,timestamp,h1_avg,h2_avg
0,14190,2014-07-28 05:42:57,NaN,NaN
1,14190,2014-07-28 05:43:01,0.377,1.280
2,14190,2014-07-28 13:41:21,0.424,1.383
3,14190,2014-07-28 13:41:24,0.362,1.669
4,14190,2014-07-28 17:43:23,0.395,1.297


In [48]:
df.count()

probeid      783990
timestamp    783990
h1_avg       781903
h2_avg       781903
dtype: int64

In [49]:
cur = con.execute('pragma foreign_keys=ON')

In [50]:
index_label = ['probeid', 'timestamp']
df = df.set_index(index_label)

In [51]:
df.head()

h1_avg  h2_avg
probeid timestamp                          
14190   2014-07-28 05:42:57     NaN     NaN
        2014-07-28 05:43:01   0.377   1.280
        2014-07-28 13:41:21   0.424   1.383
        2014-07-28 13:41:24   0.362   1.669
        2014-07-28 17:43:23   0.395   1.297

In [54]:
df.to_sql(  '%s'%SK_LATENCIES_AGG
          , con
          , flavor='sqlite'
          , if_exists = 'append'
          , index_label = index_label
         )

In [55]:
con.commit()
con.close()